In [1]:
import numpy as np
import pandas as pd

# cleaning

In [2]:
data = pd.read_csv('C:/Users/XBOX2/Desktop/stochastic project/Dow Jones Industrial Average Historical Data.csv')
df = pd.DataFrame(data)

In [3]:
df.head(5)

,Date,Price,Open,High,Low,Vol.,Change %
0,11/28/2025,"47,716.42","47,482.25","47,750.77","47,475.61",272.51M,0.61%
1,11/26/2025,"47,427.12","47,196.15","47,571.40","47,196.15",458.14M,0.67%
2,11/25/2025,"47,112.45","46,482.36","47,182.90","46,341.35",659.61M,1.43%
3,11/24/2025,"46,448.27","46,351.93","46,587.71","46,108.01",748.69M,0.44%
4,11/21/2025,"46,245.41","45,808.65","46,577.50","45,781.58",795.91M,1.08%


In [4]:
# convert the data from string to float
for col in ['Price','Open', 'High', 'Low']:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)
df['Change %'] = df['Change %'].astype(str).str.replace('%','').astype(float)

In [5]:
lst = []
for num in df['Vol.']:
    num = str(num)
    if 'B' in num:
        val = float(num.replace('B','')) * 1e9
        lst.append(val)
    elif 'M' in num:
        val = float(num.replace('M','')) * 1e6
        lst.append(val)
    else:
        lst.append(float(num))

df['Vol.'] = lst

In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
# sort date
df = df.sort_values(by='Date').reset_index(drop=True)


In [8]:
df

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-01-02,37715.04,37566.22,37790.08,37495.91,350290000.0,0.07
1,2024-01-03,37430.19,37629.23,37629.23,37401.85,329140000.0,-0.76
2,2024-01-04,37440.34,37425.28,37716.41,37425.28,380220000.0,0.03
3,2024-01-05,37466.11,37455.46,37623.62,37323.82,299490000.0,0.07
4,2024-01-08,37683.01,37327.37,37692.92,37249.24,362200000.0,0.58
...,...,...,...,...,...,...,...
476,2025-11-21,46245.41,45808.65,46577.50,45781.58,795910000.0,1.08
477,2025-11-24,46448.27,46351.93,46587.71,46108.01,748690000.0,0.44
478,2025-11-25,47112.45,46482.36,47182.90,46341.35,659610000.0,1.43
479,2025-11-26,47427.12,47196.15,47571.40,47196.15,458140000.0,0.67


##  detect the states firstly


In [22]:
df['returns'] = df['Price'].pct_change() # to detect the Bull ,Bear market
df['range'] = (df['High'] - df['Low']) / df['Open'] # to detect the volatility of the stock
df['vol_chg'] = df['Vol.'].pct_change() 

C:\Users\XBOX2\AppData\Local\Temp\ipykernel_15220\1697799872.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['vol_chg'] = df['Vol.'].pct_change()


In [23]:
df.isna().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Vol.        1
Change %    0
returns     1
range       0
vol_chg     1
dtype: int64

### explain the function of pct_change 
#### to measure the diff between 2 rows (current value - pervious value ) / pervious value

In [24]:
df['vol_chg'] = df['vol_chg'].fillna(0)
df['Vol.'] = df['Vol.'].fillna(0)
df['returns'] = df['returns'].fillna(0)

In [25]:
df.isna().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
returns     0
range       0
vol_chg     0
dtype: int64

## steps for apply Baum witch algorithm

In [26]:
df.shape

(481, 10)

In [ ]:
# intialize the parameters  is randomly before training the HMM model
N_states = 3
T = df.shape[0]
pi = np.ones(N_states)/N_states 
A = np.ones((N_states,N_states))/N_states 
obs_dim = 3
B_mean = np.random.rand(N_states, obs_dim) 
B_cov = np.array([np.eye(obs_dim) for _ in range(N_states)])

In [32]:
B_mean

array([[0.40043349, 0.79828107, 0.00579974],
       [0.49340085, 0.49331681, 0.9413853 ],
       [0.12506096, 0.752547  , 0.26786943]])

In [33]:
from scipy.stats import multivariate_normal

In [34]:
def forward(obs, pi, A, B_mean, B_cov):
    T = obs.shape[0]
    N = pi.shape[0]
    alpha = np.zeros((T,N))
    
    # Initialization
    for i in range(N):
        alpha[0,i] = pi[i] * multivariate_normal.pdf(obs[0], mean=B_mean[i], cov=B_cov[i])
    
    # Recursion
    for t in range(1, T):
        for j in range(N):
            alpha[t,j] = np.sum(alpha[t-1,:] * A[:,j]) * multivariate_normal.pdf(obs[t], mean=B_mean[j], cov=B_cov[j])
    return alpha


In [35]:
def backward(obs, A, B_mean, B_cov):
    T = obs.shape[0]
    N = A.shape[0]
    beta = np.zeros((T,N))
    beta[T-1,:] = 1  # Initialization
    
    for t in range(T-2, -1, -1):
        for i in range(N):
            beta[t,i] = np.sum(A[i,:] * [multivariate_normal.pdf(obs[t+1], mean=B_mean[j], cov=B_cov[j]) for j in range(N)] * beta[t+1,:])
    return beta

In [36]:



def baum_welch(obs, A_init, B_mean_init, B_cov_init, pi_init, max_iter=100, tol=1e-4):
    T = obs.shape[0]
    N = A_init.shape[0]
    A, B_mean, B_cov, pi = A_init.copy(), B_mean_init.copy(), B_cov_init.copy(), pi_init.copy()
    log_likelihoods = []

    for iteration in range(max_iter):
       

        alpha = forward(obs, pi, A, B_mean, B_cov)
        beta = backward(obs, A, B_mean, B_cov)

        prob_O = np.sum(alpha[T-1, :])
        log_likelihood = np.log(prob_O)
        log_likelihoods.append(log_likelihood)

        if iteration > 0 and (log_likelihood - log_likelihoods[-2]) < tol:
            break

        gamma = (alpha * beta) / prob_O

        xi = np.zeros((T - 1, N, N))
        for t in range(T - 1):
            for i in range(N):
                for j in range(N):
                    B_emission = multivariate_normal.pdf(obs[t+1], mean=B_mean[j], cov=B_cov[j])
                    xi[t, i, j] = alpha[t, i] * A[i, j] * B_emission * beta[t+1, j]
            sum_xi_t = np.sum(xi[t, :, :])
            if sum_xi_t > 0:
                 xi[t, :, :] /= sum_xi_t


       

       
       
        pi = gamma[0, :]

      
       
        sum_xi_t = np.sum(xi, axis=0) 
        sum_gamma_t = np.sum(gamma[:-1, :], axis=0) 
        
        A = sum_xi_t / sum_gamma_t[:, np.newaxis]
       
        A = A / np.sum(A, axis=1)[:, np.newaxis]

        
        for j in range(N):
            
            
            weighted_obs_sum = np.sum(gamma[:, j, np.newaxis] * obs, axis=0)

            
            
            sum_gamma_j = np.sum(gamma[:, j])

           
            B_mean[j] = weighted_obs_sum / sum_gamma_j

           
            
            diff = obs - B_mean[j] 
            weighted_cov_sum = np.dot((gamma[:, j, np.newaxis] * diff).T, diff)
            B_cov[j] = weighted_cov_sum / sum_gamma_j
            
           


    return A, B_mean, B_cov, pi, log_likelihoods

In [37]:
df.to_csv('C:/Users/XBOX2/Desktop/stochastic project/Dow_Jones_Industrial_Average_Historical_Data_Cleaned.csv', index=False)

In [38]:
import numpy as np
from scipy.stats import multivariate_normal

# --- 1. Scaled Forward Algorithm (للاستقرار العددي) ---
def forward_scaled(obs, pi, A, B_mean, B_cov):
    T = obs.shape[0]
    N = pi.shape[0]
    alpha_hat = np.zeros((T,N))
    c = np.zeros(T) # عوامل القياس

    # Initialization
    B_init = np.array([multivariate_normal.pdf(obs[0], mean=B_mean[i], cov=B_cov[i]) for i in range(N)])
    alpha_hat[0,:] = pi * B_init
    c[0] = np.sum(alpha_hat[0,:])
    if c[0] > 0:
        alpha_hat[0,:] /= c[0]

    # Recursion
    for t in range(1, T):
        B_t = np.array([multivariate_normal.pdf(obs[t], mean=B_mean[j], cov=B_cov[j]) for j in range(N)])
        for j in range(N):
            alpha_hat[t,j] = np.sum(alpha_hat[t-1,:] * A[:,j]) * B_t[j]

        # Scaling
        c[t] = np.sum(alpha_hat[t,:])
        if c[t] > 0:
            alpha_hat[t,:] /= c[t]
        else:
             alpha_hat[t,:] = 0

    return alpha_hat, c

# --- 2. Scaled Backward Algorithm (للاستقرار العددي) ---
def backward_scaled(obs, A, B_mean, B_cov, c):
    T = obs.shape[0]
    N = A.shape[0]
    beta_hat = np.zeros((T,N))
    
    # Initialization
    beta_hat[T-1,:] = 1 / c[T-1] if c[T-1] > 0 else 1

    # Recursion
    for t in range(T-2, -1, -1):
        B_emissions = np.array([multivariate_normal.pdf(obs[t+1], mean=B_mean[j], cov=B_cov[j]) for j in range(N)])
        for i in range(N):
            beta_hat[t,i] = np.sum(A[i,:] * B_emissions * beta_hat[t+1,:])
        
        # Scaling
        if c[t] > 0:
            beta_hat[t,:] /= c[t]
        else:
            beta_hat[t,:] = 0

    return beta_hat

# --- 3. Scaled Baum-Welch Algorithm (لإعادة تقدير المعلمات) ---
def baum_welch_scaled(obs, A_init, B_mean_init, B_cov_init, pi_init, max_iter=50, tol=1e-5):
    T = obs.shape[0]
    N = A_init.shape[0]
    D_OBS = obs.shape[1]
    A, B_mean, B_cov, pi = A_init.copy(), B_mean_init.copy(), B_cov_init.copy(), pi_init.copy()
    log_likelihoods = []

    for iteration in range(max_iter):
        # E-Step: Calculate Forward, Backward, Log-Likelihood
        alpha_hat, c = forward_scaled(obs, pi, A, B_mean, B_cov)
        beta_hat = backward_scaled(obs, A, B_mean, B_cov, c)
        
        log_likelihood = np.sum(np.log(c[np.where(c > 0)]))
        log_likelihoods.append(log_likelihood)

        if iteration > 0 and abs(log_likelihood - log_likelihoods[-2]) < tol:
            break

        # Compute Gamma and Xi (باستخدام القيم المقيسة)
        gamma = alpha_hat * beta_hat
        gamma = np.nan_to_num(gamma / np.sum(gamma, axis=1)[:, np.newaxis], nan=0) # Normalization

        xi = np.zeros((T - 1, N, N))
        for t in range(T - 1):
            B_emissions_t1 = np.array([multivariate_normal.pdf(obs[t+1], mean=B_mean[j], cov=B_cov[j]) for j in range(N)])
            for i in range(N):
                for j in range(N):
                    xi[t, i, j] = alpha_hat[t, i] * A[i, j] * B_emissions_t1[j] * beta_hat[t+1, j]

            sum_xi_t = np.sum(xi[t, :, :])
            if sum_xi_t > 0:
                 xi[t, :, :] /= sum_xi_t
            else:
                 xi[t, :, :] = 0

        # M-Step: Re-estimation
        pi = gamma[0, :] / np.sum(gamma[0, :])

        sum_xi_t = np.sum(xi, axis=0)
        sum_gamma_t = np.sum(gamma[:-1, :], axis=0)
        
        A = np.zeros((N, N))
        for i in range(N):
            if sum_gamma_t[i] > 1e-12:
                A[i, :] = sum_xi_t[i, :] / sum_gamma_t[i]
            else:
                A[i, :] = 1.0 / N
        A = A / np.sum(A, axis=1)[:, np.newaxis]

        for j in range(N):
            sum_gamma_j = np.sum(gamma[:, j])
            
            if sum_gamma_j > 1e-12:
                weighted_obs_sum = np.sum(gamma[:, j, np.newaxis] * obs, axis=0)
                B_mean[j] = weighted_obs_sum / sum_gamma_j

                diff = obs - B_mean[j]
                weighted_cov_sum = np.dot((gamma[:, j, np.newaxis] * diff).T, diff)
                B_cov[j] = weighted_cov_sum / sum_gamma_j

            B_cov[j] += np.eye(D_OBS) * 1e-6 # Regularization

    return A, B_mean, B_cov, pi, log_likelihoods

# --- 4. Log-Viterbi Algorithm (لإخراج تسلسل الحالات المخفية) ---
def viterbi_log(obs, A, B_mean, B_cov, pi):
    """خوارزمية فيتربي لإيجاد المسار الأكثر ترجيحًا للحالات المخفية."""
    T = obs.shape[0]
    N = A.shape[0]
    
    log_A = np.log(A + 1e-100)
    log_pi = np.log(pi + 1e-100)
    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)
    
    def log_B_t(t, j):
        try:
            # logpdf هي الدالة اللوغاريتمية لـ multivariate_normal.pdf
            return multivariate_normal.logpdf(obs[t], mean=B_mean[j], cov=B_cov[j])
        except np.linalg.LinAlgError:
            return -np.inf

    # Initialization
    for i in range(N):
        delta[0, i] = log_pi[i] + log_B_t(0, i)

    # Recursion
    for t in range(1, T):
        for j in range(N):
            # نستخدم log(a+b) = log(a) + log(b) بدلاً من الضرب
            log_transition_probs = delta[t-1, :] + log_A[:, j]
            
            max_log_prob_prev = np.max(log_transition_probs)
            max_idx_prev = np.argmax(log_transition_probs)
            
            delta[t, j] = max_log_prob_prev + log_B_t(t, j)
            psi[t, j] = max_idx_prev

    # Path Backtracking
    Q = np.zeros(T, dtype=int)
    Q[T-1] = np.argmax(delta[T-1, :])

    for t in range(T - 2, -1, -1):
        Q[t] = psi[t+1, Q[t+1]]

    return Q

In [42]:
df[['range', 'returns', 'vol_chg']].shape

(481, 3)

In [43]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans




obs_features = ['returns','range','vol_chg']
obs = df[obs_features].values
N_STATES = 3
D_OBS = obs.shape[1]

# --- 2. تهيئة المعلمات الابتدائية (Initial Parameters) ---
# يجب أن تبدأ بنماذج تخمين أولية، غالبًا ما نستخدم التوزيع العشوائي أو K-Means.
np.random.seed(42)

# تهيئة احتمالات البداية (pi)
pi_init = np.random.rand(N_STATES)
pi_init = pi_init / np.sum(pi_init)

# تهيئة مصفوفة الانتقال (A)
A_init = np.random.rand(N_STATES, N_STATES)
A_init = A_init / np.sum(A_init, axis=1)[:, np.newaxis]

# تهيئة متوسطات الانبعاث (B_mean) باستخدام تجميع K-Means
kmeans = KMeans(n_clusters=N_STATES, random_state=42, n_init='auto')
kmeans.fit(obs)
B_mean_init = kmeans.cluster_centers_

# تهيئة مصفوفات التغاير (B_cov)
B_cov_init = np.array([np.diag(np.diag(np.cov(obs.T))) * 0.5 for _ in range(N_STATES)])
B_cov_init += np.eye(D_OBS) * 1e-6


# --- 3. تدريب النموذج (خوارزمية بوم-ويلش) ---
# هذه الخطوة تجد أفضل المعلمات (A, B_mean, B_cov, pi)
print("--- بدء التدريب (Baum-Welch) ---")
# افترض أن baum_welch_scaled هي دالتك المستقرة عددياً
# (يجب تعريفها في الكود قبل هذه النقطة)
A_final, B_mean_final, B_cov_final, pi_final, log_likes = baum_welch_scaled(
    obs, A_init, B_mean_init, B_cov_init, pi_init, max_iter=50
)
print("--- انتهى التدريب ---")


# --- 4. تحديد تسلسل الحالات المخفية (خوارزمية فيتربي) ---
# هذه الخطوة هي التي تولد لك قائمة (مصفوفة) الحالات لكل صف
print("--- تحديد تسلسل الحالات (Viterbi) ---")
# افترض أن viterbi_log هي دالتك المستقرة عددياً
hidden_states = viterbi_log(obs, A_final, B_mean_final, B_cov_final, pi_final)


# --- 5. تطبيق تسلسل الحالات على الـ DataFrame ---

# إنشاء العمود الجديد باسم 'Hidden_State' (يحتوي على الأرقام 0, 1, 2)
df['Hidden_State'] = hidden_states 

# إضافة أسماء وصفية للحالات بناءً على متوسط العائد لكل حالة
mean_returns = B_mean_final[:, 0] # العائدات هي السمة الأولى (index 0)
sort_idx = np.argsort(mean_returns)
state_map = {
    sort_idx[0]: 'Bear Market (هبوطي)',       # الحالة ذات أقل متوسط عائد
    sort_idx[1]: 'Stagnant (ركود)',   # الحالة ذات متوسط العائد المتوسط
    sort_idx[2]: 'Bull Market (صعودي)'        # الحالة ذات أعلى متوسط عائد
}
df['Market_Regime'] = df['Hidden_State'].map(state_map)

print("\n✅ تم تطبيق الحالات المخفية بنجاح.")
print("أول 5 صفوف مع الحالات الجديدة:")
print(df[['returns', 'range', 'Hidden_State', 'Market_Regime']].head())

--- بدء التدريب (Baum-Welch) ---
--- انتهى التدريب ---
--- تحديد تسلسل الحالات (Viterbi) ---

✅ تم تطبيق الحالات المخفية بنجاح.
أول 5 صفوف مع الحالات الجديدة:
    returns     range  Hidden_State        Market_Regime
0  0.000000  0.007831             1  Bull Market (صعودي)
1 -0.007553  0.006043             1  Bull Market (صعودي)
2  0.000271  0.007779             1  Bull Market (صعودي)
3  0.000688  0.008004             1  Bull Market (صعودي)
4  0.005789  0.011886             1  Bull Market (صعودي)
--- انتهى التدريب ---
--- تحديد تسلسل الحالات (Viterbi) ---

✅ تم تطبيق الحالات المخفية بنجاح.
أول 5 صفوف مع الحالات الجديدة:
    returns     range  Hidden_State        Market_Regime
0  0.000000  0.007831             1  Bull Market (صعودي)
1 -0.007553  0.006043             1  Bull Market (صعودي)
2  0.000271  0.007779             1  Bull Market (صعودي)
3  0.000688  0.008004             1  Bull Market (صعودي)
4  0.005789  0.011886             1  Bull Market (صعودي)


In [45]:
df.head(10)

,Date,Price,Open,High,Low,Vol.,Change %,returns,range,vol_chg,Hidden_State,Market_Regime
0,2024-01-02,37715.04,37566.22,37790.08,37495.91,350290000.0,0.07,0.000000,0.007831,0.000000,1,Bull Market (صعودي)
1,2024-01-03,37430.19,37629.23,37629.23,37401.85,329140000.0,-0.76,-0.007553,0.006043,-0.060379,1,Bull Market (صعودي)
2,2024-01-04,37440.34,37425.28,37716.41,37425.28,380220000.0,0.03,0.000271,0.007779,0.155192,1,Bull Market (صعودي)
3,2024-01-05,37466.11,37455.46,37623.62,37323.82,299490000.0,0.07,0.000688,0.008004,-0.212324,1,Bull Market (صعودي)
4,2024-01-08,37683.01,37327.37,37692.92,37249.24,362200000.0,0.58,0.005789,0.011886,0.209389,1,Bull Market (صعودي)
5,2024-01-09,37525.16,37523.55,37552.38,37373.30,293230000.0,-0.42,-0.004189,0.004772,-0.190420,1,Bull Market (صعودي)
6,2024-01-10,37695.73,37552.91,37740.77,37524.40,279540000.0,0.45,0.004545,0.005762,-0.046687,1,Bull Market (صعودي)
7,2024-01-11,37711.02,37747.14,37801.90,37424.28,305800000.0,0.04,0.000406,0.010004,0.093940,1,Bull Market (صعودي)
8,2024-01-12,37592.98,37818.05,37825.27,37470.19,279260000.0,-0.31,-0.003130,0.009389,-0.086789,1,Bull Market (صعودي)
9,2024-01-16,37361.12,37493.54,37543.18,37201.39,379980000.0,-0.62,-0.006168,0.009116,0.360667,1,Bull Market (صعودي)


In [48]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [49]:
features_to_plot = ['returns', 'range', 'vol_chg', 'Low', 'Vol.']

# 4. إنشاء مخططات الكمان (Violin Plots) لكل ميزة
fig = make_subplots(
    rows=len(features_to_plot), cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=[f"Distribution of {feature} by Market Regime" for feature in features_to_plot]
)

color_map = {'Bull Market (صعودي)': 'green', 'Bear Market (هبوطي)': 'red', 'Stagnant (ركود)': 'blue'}
unique_regimes = df['Market_Regime'].unique()

for i, feature in enumerate(features_to_plot):
    row_index = i + 1
    
    for regime in unique_regimes:
        color = color_map.get(regime, 'black')
        
        fig.add_trace(
            go.Violin(
                y=df[df['Market_Regime'] == regime][feature],
                name=regime,
                box_visible=True,
                meanline_visible=True,
                line_color=color,
                fillcolor=color,
                opacity=0.6,
                legendgroup=regime,
                showlegend=(i == 0)
            ),
            row=row_index, col=1
        )
    
    fig.update_yaxes(title_text=feature, row=row_index, col=1, zeroline=True)

fig.update_layout(
    title_text="Violin Plots of Key Features Grouped by HMM Market Regime",
    height=400 * len(features_to_plot),
    showlegend=True,
    xaxis_title="Market Regime"
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'box': {'visible': True},
              'fillcolor': 'green',
              'legendgroup': 'Bull Market (صعودي)',
              'line': {'color': 'green'},
              'meanline': {'visible': True},
              'name': 'Bull Market (صعودي)',
              'opacity': 0.6,
              'showlegend': True,
              'type': 'violin',
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAAAAAGs6+ke9+vwDQ0qKAxT' ... 'cZ73s/ADnFSpJbez8AF2aJMfx4Pw=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'box': {'visible': True},
              'fillcolor': 'red',
              'legendgroup': 'Bear Market (هبوطي)',
              'line': {'color': 'red'},
              'meanline': {'visible': True},
              'name': 'Bear Market (هبوطي)',
              'opacity': 0.6,
              'showlegend': True,
              'type': 'violin',
              'xaxis': 'x',
              'y': {'bdata': ('QOgBCZWxi7+AsAd97r+LvwBgOWfSVo' ... 'r2JBOGPwDfAk2u93E/AAtUEfpIjT8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'box': {'visible': True},
              'fillcolor': 'blue',
              'legendgroup': 'Stagnant (ركود)',
              'line': {'color': 'blue'},
              'meanline': {'visible': True},
              'name': 'Stagnant (ركود)',
              'opacity': 0.6,
              'showlegend': True,
              'type': 'violin',
              'xaxis': 'x',
              'y': {'bdata': ('AOwjXyLtUz+A5xhI0hh0vwAhVgHm44' ... 'rX4FFLPwAcXpJ/aYQ/AJgmMfSvbj8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'box': {'visible': True},
              'fillcolor': 'green',
              'legendgroup': 'Bull Market (صعودي)',
              'line': {'color': 'green'},
              'meanline': {'visible': True},
              'name': 'Bull Market (صعودي)',
              'opacity': 0.6,
              'showlegend': False,
              'type': 'violin',
              'xaxis': 'x2',
              'y': {'bdata': ('fbkfe4sJgD+4SB6bK8B4Pzf0k2zW3H' ... 'Luu4E/zGoYa4pIgD/bm4KCgbx3Pw=='),
                    'dtype': 'f8'},
              'yaxis': 'y2'},
             {'box': {'visible': True},
              'fillcolor': 'red',
              'legendgroup': 'Bear Market (هبوطي)',
              'line': {'color': 'red'},
              'meanline': {'visible': True},
              'name': 'Bear Market (هبوطي)',
              'opacity': 0.6,
              'showlegend': False,
              'type': 'violin',
              'xaxis': 'x2',
              'y': {'bdata': ('ZUKtQhhykT/+2QOj/G6WP8wvA5BRnY' ... 'rBuMqRP+6F433mMYU/0GiLBwuKkj8='),
                    'dtype': 'f8'},
              'yaxis': 'y2'},
             {'box': {'visible': True},
              'fillcolor': 'blue',
              'legendgroup': 'Stagnant (ركود)',
              'line': {'color': 'blue'},
              'meanline': {'visible': True},
              'name': 'Stagnant (ركود)',
              'opacity': 0.6,
              'showlegend': False,
              'type': 'violin',
              'xaxis': 'x2',
              'y': {'bdata': ('ND5ulWXOez/MItto6VqAPyL+0LiR1p' ... 'b3S5KAPyrd+7bCsIU/aTuHbUjVeT8='),
                    'dtype': 'f8'},
              'yaxis': 'y2'},
             {'box': {'visible': True},
              'fillcolor': 'green',
              'legendgroup': 'Bull Market (صعودي)',
              'line': {'color': 'green'},
              'meanline': {'visible': True},
              'name': 'Bull Market (صعودي)',
              'opacity': 0.6,
              'showlegend': False,
              'type': 'violin',
              'xaxis': 'x3',
              'y': {'bdata': ('AAAAAAAAAAAgcVy77+muv5ixKIhX3c' ... 'v1LLo/0v6vFUyM07+sd8S8f+7Zvw=='),
                    'dtype': 'f8'},
              'yaxis': 'y3'},
             {'box': {'visible': True},
              'fillcolor': 'red',
              'legendgroup': 'Bear Market (هبوطي)',
              'line': {'